# 요약 (Summarization )
`14_summarization.ipynb`

매우 많은 양의 컨텍스트가 있을 경우, 어떻게 요약을 해야 할까?
1. 프롬프트에 다 때려 박기
2. Map-Reduce: 각 문서를 요약하고, 이것들을 다 합쳐서 최종 요약본을 만든다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
%pip install -q arxiv

In [ ]:
from langchain_community.document_loaders import WebBaseLoader, ArxivLoader

loader = ArxivLoader(
    query='reasoning',
    load_max_docs=2,
)

docs = loader.load()
docs = docs[:2]

[Document(metadata={'Published': '2025-04-01', 'Title': 'Process or Result? Manipulated Ending Tokens Can Mislead Reasoning LLMs to Ignore the Correct Reasoning Steps', 'Authors': 'Yu Cui, Bryan Hooi, Yujun Cai, Yiwei Wang', 'Summary': 'Recent reasoning large language models (LLMs) have demonstrated remarkable\nimprovements in mathematical reasoning capabilities through long\nChain-of-Thought. The reasoning tokens of these models enable self-correction\nwithin reasoning chains, enhancing robustness. This motivates our exploration:\nhow vulnerable are reasoning LLMs to subtle errors in their input reasoning\nchains? We introduce "Compromising Thought" (CPT), a vulnerability where models\npresented with reasoning tokens containing manipulated calculation results tend\nto ignore correct reasoning steps and adopt incorrect results instead. Through\nsystematic evaluation across multiple reasoning LLMs, we design three\nincreasingly explicit prompting methods to measure CPT resistance, revea

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1', temperature=0)

## 문서 때려 박기 (Stuff Docs)

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', '아래 내용을 정확하게 요약해: \n\n{context}')
])

chain = create_stuff_documents_chain(llm, prompt)

res = chain.invoke({'context': docs})

In [8]:
from pprint import pprint

print(res)

아래 두 논문의 핵심 내용을 각각 정확하게 요약합니다.

---

### 1. **Process or Result? Manipulated Ending Tokens Can Mislead Reasoning LLMs to Ignore the Correct Reasoning Steps**

**핵심 요약:**

- **연구 목적:** 최근 수리 추론에 강한 LLM(대형 언어모델)들은 Chain-of-Thought(CoT) 방식으로 자기 검증 및 오류 수정 능력이 향상되었으나, 입력된 추론 과정(Reasoning Tokens)에 미세한 오류가 포함될 경우 얼마나 취약한지 분석함.
- **주요 발견:** 
  - **Compromising Thought (CPT) 현상:** 추론 과정의 마지막 계산 결과(ending tokens)만 살짝 조작해도, LLM은 올바른 추론 과정을 무시하고 조작된(틀린) 결과를 최종 답으로 채택하는 경향이 강함.
  - **실험 결과:** 여러 LLM(DeepSeek-R1, OpenAI-o1, o3-mini 등)에서 CPT 저항성을 세 가지 방법(불확실성 유도, 명시적 오류 지시, 출력 프리픽스 강제)으로 측정했으나, 대부분 모델이 조작된 결과를 잘못 받아들이고 자기 수정에 실패함.
  - **기존 연구와의 차이:** 기존에는 추론 구조(구성) 변화가 내용(결과) 변화보다 모델 성능에 더 큰 영향을 준다고 했으나, 본 연구는 마지막 결과 토큰의 미세 조작이 구조 변화보다 더 치명적임을 보임.
  - **보안 취약점:** DeepSeek-R1은 조작된 reasoning tokens가 입력되면 아예 추론을 멈추고 답을 내지 않는 현상(“thinking stopped”)이 발생함.
- **의의:** LLM의 자기 수정 및 추론 견고성에 심각한 취약점이 있음을 밝히고, 수리/추론 중심 응용에서 보안상 주의가 필요함을 강조함.

---

### 2. **Hypothesis Testing Prompting Improves Deductive Reasoni